Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
import datetime

In [2]:
#for cleaning data

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from nltk.stem import PorterStemmer
import re

stop_words=stopwords.words('english')
punct=string.punctuation
stemmer=PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xzono\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#for making model

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
# for saving the model

from joblib import dump, load

Data (source: https://www.kaggle.com/kazanova/sentiment140?select=training.1600000.processed.noemoticon.csv)

In [5]:
tweets = pd.read_csv('../lotweets.csv', names=['polarity', 'um', 'so-and-so', 'something', 'trash', 'tweet'])
tweets

FileNotFoundError: [Errno 2] File ../lotweets.csv does not exist: '../lotweets.csv'

In [ ]:
#clean the text
cleets = []
for twittre in tweets.tweet:
    #this removes mentions
    twittre = re.sub(r'@[A-Za-z0-9_]+', '', twittre)
    #this removes urls
    twittre = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', twittre)        
    #this removes everything but letters from the str
    twittre = re.sub('[^a-zA-Z]',' ',twittre)
    #this makes everything lowercase, then turns everything that is separated by a space into its own str
    twittre = twittre.lower().split()
    #this removes stop words and stems everything else
    twittre = [stemmer.stem(word) for word in twittre if (word not in stop_words)]
    # this puts it all back together with spaces inbetween
    twittre = ' '.join(twittre)
    cleets.append(twittre)
    if not len(cleets)%10000:
        print(len(cleets))

In [ ]:
tweets = pd.DataFrame({'Polarity':tweets.polarity, 'Tweet':cleets})
tweets.to_csv('../lotwean.csv')

Cleaning words from https://www.kaggle.com/nltkdata/opinion-lexicon

In [ ]:
negative = pd.read_csv('../negative-words.txt', names=['words'])
positive = pd.read_csv('../positive-words.txt', names=['words'])
cleets = []
for twittre in negative.words:
    #this removes mentions
    twittre = re.sub(r'@[A-Za-z0-9_]+', '', twittre)
    #this removes urls
    twittre = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', twittre)        
    #this removes everything but letters from the str
    twittre = re.sub('[^a-zA-Z]',' ',twittre)
    #this makes everything lowercase, then turns everything that is separated by a space into its own str
    twittre = twittre.lower().split()
    #this removes stop words and stems everything else
    twittre = [stemmer.stem(word) for word in twittre if (word not in stop_words)]
    # this puts it all back together with spaces inbetween
    twittre = ' '.join(twittre)
    cleets.append(twittre)
negative = set(cleets)
with open('../negatean.txt', 'w+') as writer:
    for word in negative:
        writer.write(word+'\n')

        
cleets = []
for twittre in positive.words:
    #this removes mentions
    twittre = re.sub(r'@[A-Za-z0-9_]+', '', twittre)
    #this removes urls
    twittre = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', twittre)        
    #this removes everything but letters from the str
    twittre = re.sub('[^a-zA-Z]',' ',twittre)
    #this makes everything lowercase, then turns everything that is separated by a space into its own str
    twittre = twittre.lower().split()
    #this removes stop words and stems everything else
    twittre = [stemmer.stem(word) for word in twittre if (word not in stop_words)]
    # this puts it all back together with spaces inbetween
    twittre = ' '.join(twittre)
    cleets.append(twittre)
positive = set(cleets)
with open('../positivean.txt', 'w+') as writer:
    for word in positive:
        writer.write(word+'\n')

Removing unnecessary words

In [ ]:
negative = set(pd.read_csv('../negativean.txt', names=['words'])['words'])
positive = set(pd.read_csv('../positivean.txt', names=['words'])['words'])
tweets = pd.read_csv('../lotwean.csv')
negativean, positivean = set([]), set([])
for n,tweet in enumerate(tweets.Tweet):
    negatweet = negative.intersection(set(tweet.split()))
    positweet = positive.intersection(set(tweet.split()))
    negativean = negativean.union(negatweet)    
    positivean = positivean.union(positweet)
    if not n%10000:
        print(n)

In [ ]:
words = negativean.union(positivean)
tweets['Twords'] = [words.intersection(set(tweet.split())) for tweet in tweets.Tweet]
tweets['Nords'] = [len(tword)!=0 for tword in tweets.Twords]

In [ ]:
tweets[['Polarity', 'Tweet', 'Twords']].to_csv('../lotwean.csv')
tweets['keep'] = [int(tword=='set()') for tword in tweets.Twords]
tweets[['Polarity', 'Tweet', 'drop']].to_csv('../tweetdb.csv', index=False)

Training the model

In [12]:
keepercentage = .4

negative = set(pd.read_csv('../negativean.txt', names=['words'])['words'])
positive = set(pd.read_csv('../positivean.txt', names=['words'])['words'])
words = list(negative.union(positive))

tweets = pd.read_csv('../tweetdb.csv')
tweets = tweets.drop(index=[i for i in tweets.index if tweets.Drop[i]])
tweets = tweets.sample(frac=keepercentage, random_state=347)
tweets['Polarity'].value_counts()

0    251095
4    239373
Name: Polarity, dtype: int64

In [13]:
vectord = CountVectorizer()
vectord.fit(words)
X = vectord.transform(tweets.Tweet).toarray()

In [14]:
X_train,X_test,y_train,y_test = train_test_split(X, tweets.Polarity,
                                                 test_size=2/7, stratify=tweets.Polarity,
                                                 shuffle=True, random_state=347)
model = MultinomialNB()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.75      0.74      0.74     71742
           4       0.73      0.74      0.74     68392

    accuracy                           0.74    140134
   macro avg       0.74      0.74      0.74    140134
weighted avg       0.74      0.74      0.74    140134



In [15]:
dump(vectord, '../vectord.joblib') 
dump(model, '../model.joblib') 

['../model.joblib']

The sentiment analyzer

In [16]:
from joblib import load

vectord = load('../vectord.joblib') 
model = load('../model.joblib') 

#tweets must be cleaned first
def sentiment(tweets, analogue=False):
    if analogue: 
        probs = model.predict_proba(vectord.transform(tweets).toarray())
        return probs[:,1]-probs[:,0]
    else: return model.predict(vectord.transform(tweets).toarray())

In [ ]:
tweets = pd.read_csv('../tweetdb.csv').sample(frac=.1, random_state=343)
print(classification_report(tweets.Polarity, sentiment(tweets.Tweet)))

In [31]:
sentiment(['Here’s the deal, even if illegals are just coming here for a “better life” they are bankrolling smugglers, cartel, and criminal organizations. If you don’t see a problem with those groups- fine, keep cheerleading illegal immigration.'])

array([4], dtype=int64)